# Verkeersdata Analyse Porfolio

In [1]:
# Libraries inladen
import pandas as pd

In [2]:
# Datasets inladen
accidents = pd.read_csv("datasets/accident_data.csv")
roads = pd.read_csv("datasets/road_data.csv")
weather = pd.read_csv("datasets/weather_data.csv")

### Onvolledige data verwijderen of aanvullen

In [3]:
def missende_waardes(df, name):
    print(f"\n Missende waarden in: {name}")
    print((df.isnull().sum() / len(df) * 100))

missende_waardes(accidents, "Accident Data")
missende_waardes(weather, "Weather Data")
missende_waardes(roads, "Road Data")


 Missende waarden in: Accident Data
ID               0.0000
Source           0.0000
Severity         0.0000
Start_Time       0.0000
End_Time         0.0000
Start_Lat        0.0000
Start_Lng        0.0000
End_Lat         44.0754
End_Lng         44.0754
Distance(mi)     0.0000
Description      0.0002
City             0.0038
County           0.0000
State            0.0000
Zipcode          0.0232
dtype: float64

 Missende waarden in: Weather Data
Start_Time            0.0000
Weather_Timestamp     1.5348
Temperature(F)        2.0932
Wind_Chill(F)        25.8034
Humidity(%)           2.2260
Pressure(in)          1.7856
Visibility(mi)        2.2582
Wind_Direction        2.2394
Wind_Speed(mph)       7.3974
Precipitation(in)    28.5232
Weather_Condition     2.2202
Sunrise_Sunset        0.2966
dtype: float64

 Missende waarden in: Road Data
ID                 0.0
Amenity            0.0
Bump               0.0
Crossing           0.0
Give_Way           0.0
Junction           0.0
No_Exit           

#### Accidents Data

In de kolommen End_lat en End_lng zitten ook heel veel missende waardes. Aangezien ik al de coordinaten van de incident locaties heb (Start_Lat - Start_Lng) heb ik deze twee kolommen niet nodig.

In [4]:
# End_Lat en End_Lng kolommen verwijderen met  veel missende waardes (indien niet nodig)
accidents.drop(['End_Lat', 'End_Lng'], axis=1, inplace=True)

In de description en city kolommen zitten heel weininig nan waardes, hierbij kunnen we heel makkelijk deze rijen veranderen, want het zal geen effect hebben op de analyse.

In [5]:
# Rijen met nan waardes verwijderen
accidents.dropna(subset=["Description", "City", "Zipcode"], inplace=True)

#### Weather Data

In de kolom 'Precipitation(in)' zitten veel missende waarden. Aangezien waarschijnlijk in de meeste geval niet regenen of sneeuwen betekent dat er geen regen of sneeuw is gevallen, besloot ik om de missende waarden met '0.0' te invullen. Zo heb ik geen dataverlies en kan werken met deze kolom.

In [6]:
# None waardes veranderen met 0
weather.fillna({"Precipitation(in)": 0.0}, inplace=True)

Als je kijkt naar de Wind_Chill(F) kolom, zie je dat het meestaal heel dichtbij of soms helemaal hetzelfde is met de Temperature(F) kolom. Dit betekent dat de NaN waardes gewoon met de waardes van Temperature(F) opgevuld kan worden. Dit geldt ook voor de kolom Temperature(F).

In [7]:
weather[["Wind_Chill(F)","Temperature(F)"]].head()

,Wind_Chill(F),Temperature(F)
0,77.0,77.0
1,43.0,45.0
2,68.0,68.0
3,15.0,27.0
4,42.0,42.0


In [8]:
# Opvullen met dichtstbijzijnde waardes
weather.fillna({"Wind_Chill(F)": "Temperature(F)"}, inplace=True)
weather.fillna({"Temperature(F)": "Wind_Chill(F)"}, inplace=True)

Omdat de resterende kolommen heel weinig NaN waardes bevatten, kunnen de rijen met de NaN values gelijk werwijderd kan worden. Dit heeft geen effect op de analyse en de data begrijpelijker.

In [9]:
# De rijen met weinig missende waardes verwijderen
weather.dropna(subset=["Humidity(%)", "Pressure(in)" 
                       ,"Visibility(mi)", "Wind_Direction", 
                       "Wind_Speed(mph)", "Weather_Condition", "Sunrise_Sunset"], 
               inplace=True)

#### Roads Data

In [10]:
weather.isnull().sum()

Start_Time           0
Weather_Timestamp    0
Temperature(F)       0
Wind_Chill(F)        0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Sunrise_Sunset       0
dtype: int64

Alle missende waardes zijn nu verwijderd.

In [11]:
missende_waardes(accidents, "Accident Data")
missende_waardes(weather, "Weather Data")
missende_waardes(roads, "Road Data")


 Missende waarden in: Accident Data
ID              0.0
Source          0.0
Severity        0.0
Start_Time      0.0
End_Time        0.0
Start_Lat       0.0
Start_Lng       0.0
Distance(mi)    0.0
Description     0.0
City            0.0
County          0.0
State           0.0
Zipcode         0.0
dtype: float64

 Missende waarden in: Weather Data
Start_Time           0.0
Weather_Timestamp    0.0
Temperature(F)       0.0
Wind_Chill(F)        0.0
Humidity(%)          0.0
Pressure(in)         0.0
Visibility(mi)       0.0
Wind_Direction       0.0
Wind_Speed(mph)      0.0
Precipitation(in)    0.0
Weather_Condition    0.0
Sunrise_Sunset       0.0
dtype: float64

 Missende waarden in: Road Data
ID                 0.0
Amenity            0.0
Bump               0.0
Crossing           0.0
Give_Way           0.0
Junction           0.0
No_Exit            0.0
Railway            0.0
Roundabout         0.0
Station            0.0
Stop               0.0
Traffic_Calming    0.0
Traffic_Signal     0.0
dtype

### Onnodige Kolommen verwijderen
Om de data overzichtelijk te houden, onnodig geheugengebruik te voorkomen en alleen met kolommen te werken die relevant zijn voor de analyse, is het handig om een paar overbodige kolommen te verwijderen.

In [12]:
# Accidents
accidents.drop(columns=[
    "Description", "Source", "Zipcode"
]
               , inplace=True)

# Weather
weather.drop(columns=[
    "Start_Time", "Wind_Chill(F)", "Visibility(mi)", "Wind_Direction"
]
            , inplace=True)

In [13]:
print(weather.columns)
print("-------------")
print(accidents.columns)

Index(['Weather_Timestamp', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition',
       'Sunrise_Sunset'],
      dtype='object')
-------------
Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'Distance(mi)', 'City', 'County', 'State'],
      dtype='object')
